# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
# TODO: Import the necessary libs
# For example: 
import os
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models.Collection import Collection
from lib.agents import Agent, AgentState
from lib.llm import LLM
from lib.state_machine import StateMachine
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
from lib.tooling import Tool
from pydantic import BaseModel
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.evaluation import AgentEvaluator, EvaluationReport
from typing import List, Optional, TypedDict, Union
import json
from tavily import TavilyClient

In [4]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")



### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created

chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

@Tool
def retrieve_game(query: str):
    """
    Tool Docstring:
    Semantic search: Finds most relevant results in the vector DB.

    Args:
        - query: a question about game industry

    Returns:
        A list of dictionaries. Each element contains:
        - Platform: e.g., Game Boy, Playstation 5, Xbox 360
        - Name: Name of the game
        - YearOfRelease: Year the game was released
        - Description: Additional details
    """

    search_results = collection.query(
        query_texts=[query],  
        n_results=15
    )

    #test search results: 
    #print("Search Results Test:", search_results.keys())
    #print("Search Results Test:")
    #print("\n")
   
    results = []
    documents = search_results["documents"][0]  
    metadatas = search_results["metadatas"][0]  

    for doc, meta in zip(documents, metadatas):
        results.append({
            "Platform": meta.get("Platform", "Unknown"),
            "Name": meta.get("Name", "Unknown"),
            "YearOfRelease": meta.get("YearOfRelease", "Unknown"),
            "Description": doc
        })
    
    return results

#results = retrieve_game("best action games")
#tools=[retrieve_game]
#print(results)

#### Evaluate Retrieval Tool

In [6]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result


@Tool
def evaluate_retrieval(question: str, retrieved_docs: list[dict]):
    """Tool Docstring:
    #    Based on the user's question and on the list of retrieved documents, 
    #    it will analyze the usability of the documents to respond to that question. 
    #    args: 
    #    - question: original question from user
    #    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    #    The result includes:
    #    - useful: whether the documents are useful to answer the question
    #    - description: description about the evaluation result
    """
    #print("Test1:", retrieved_docs)
    #print("Test2:", type(retrieved_docs))
    
    #modified this as I found that retrieved docs when passed is a list of strings and not a dic. 
    context = "\n\n".join(retrieved_docs)
    
    agent = Agent(
        query = question,
        model_name="gpt-4o-mini",
        tools=[retrieve_game],
        instructions=f"""
            Evaluate if the following documents are sufficient to answer the question.
            Question: "{question}"
            Documents: {context}
            Return only valid JSON: {{ "useful": true/false, "description": "..." }}
            """
        )

    result = agent.invoke(question)

    evaluator = AgentEvaluator()

    # Get final state and response. Referenced m10_demo from Udemy workspace / course info. 
    final_state:AgentState = result.get_final_state()
    if final_state and final_state.get("messages"):
        # Find the last AI message as the final response
        final_response = ""
        for msg in reversed(final_state["messages"]):
            if isinstance(msg, AIMessage) and msg.content:
                final_response = msg.content
                break
       # print("Test", final_response)


    report = json.loads(final_response)

    
    return report

#print(evaluate_retrieval("When was gran turismo 5 released", results))

#### Game Web Search Tool

In [7]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web

@Tool
def game_web_search(question: str):
    """
    Tool Docstring:
    Semantic search: Finds most relevant results from the web.
    
    Args:
        - question: a question about the game industry
    
    Returns:
        A list of search results. Each element contains:
        - Title: Title of the webpage or article
        - URL: Link to the page
        - Snippet: Short description or excerpt
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY )

    results = tavily_client.search(query=question)  
    print("type: ", type(results))
    print("test: ", results)
     
    formatted_results = []
    #used chatgpt for troubleshooting. Helped me realize r.get was not working as i was iterating over top level dict instead the contents inside. 
    for r in results["results"]:
        print(json.dumps(r, indent=2))
        formatted_results.append({
            "Title": r.get("title", "Unknown"),
            "URL": r.get("url", "Unknown"),
            "Snippet": r.get("snippet", "")
        })
    
    return formatted_results
    
#print(game_web_search("When was gran turismo 5 released"))

### Agent

In [8]:
tools = [retrieve_game ,evaluate_retrieval, game_web_search]

In [9]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

# Used / referenced Udacity's Implemented Agent State with a Task Flow content to create agent and implement exisiting tools 
class Agent:
    def __init__(self,
                query: str,
                model_name: str,
                instructions: str,
                retrieved_docs: list = [],
                evaluation_result: str = "",
                web_results: list = [],
                final_answer: str = "",
                tools: List = None):
        """
        Initialize an Agent instance
        
        Args:
            model_name: Name/identifier of the LLM model to use
            instructions: System instructions for the agent
            tools: Optional list of tools available to the agent
            
        """
        self.query = query
        self.model_name = model_name
        self.instructions = instructions
        self.messages: list = []
        self.retrieved_docs = retrieved_docs
        self.evaluation_result = evaluation_result
        self.web_results = web_results
        self.final_answer = final_answer
        self.tools = tools or []
                
        # Initialize state machine
        self.workflow = self._create_state_machine()

    def _prepare_messages_step(self, state: AgentState) -> AgentState:
        """Step logic: Prepare messages for LLM consumption"""

        messages = [
            SystemMessage(content=state["instructions"]),
            UserMessage(content=state["query"])
        ]
        
        return {
            "messages": messages
        }

    def _llm_step(self, state: AgentState) -> AgentState:
        """Step logic: Process the current state through the LLM"""

        # Initialize LLM
        llm = LLM(
            model=self.model_name,
            tools=self.tools
        )

        response = llm.invoke(state["messages"])

        tool_calls = response.tool_calls if response.tool_calls else None
        

        # Create AI message with content and tool calls
        ai_message = AIMessage(content=response.content, tool_calls=tool_calls)
        #print("Tool Calls in LLm Step\n", ai_message,"\n")
        
        return {
            "messages": state["messages"] + [ai_message],
            "current_tool_calls": tool_calls
        }

    def _tool_step(self, state: AgentState) -> AgentState:
        """Step logic: Execute any pending tool calls"""
        tool_calls = state["current_tool_calls"] or []
        tool_messages = []
        
        
        for call in tool_calls:
            # Access tool call data correctly
           
            function_name = call.function.name
            
            #function_args = json.loads(call.function.arguments)
           
            raw_args = call.function.arguments

            if raw_args:
                function_args = json.loads(raw_args)
            else:
                function_args = {}

                
            tool_call_id = call.id
            
            # Find the matching tool
            tool = next((t for t in self.tools if t.name == function_name), None)
            if tool:
                result = tool(**function_args)
                tool_messages.append(
                    ToolMessage(
                        content=json.dumps(result), 
                        tool_call_id=tool_call_id, 
                        name=function_name, 
                    )
                )
        
        # Clear tool calls and add results to messages
        return {
            "messages": state["messages"] + tool_messages,
            "current_tool_calls": None
        }

    def _create_state_machine(self) -> StateMachine[AgentState]:
        """Create the internal state machine for the agent"""
        machine = StateMachine[AgentState](AgentState)
        
        # Create steps
        entry = EntryPoint[AgentState]()
        message_prep = Step[AgentState]("message_prep", self._prepare_messages_step)
        llm_processor = Step[AgentState]("llm_processor", self._llm_step)
        tool_executor = Step[AgentState]("tool_executor", self._tool_step)
        termination = Termination[AgentState]()
        
        machine.add_steps([entry, message_prep, llm_processor, tool_executor, termination])
        
        # Add transitions
        machine.connect(entry, message_prep)
        machine.connect(message_prep, llm_processor)
        
        # Transition based on whether there are tool calls
        def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
            """Transition logic: Check if there are tool calls"""
            if state.get("current_tool_calls"):
                return tool_executor
            return termination
        
        machine.connect(llm_processor, [tool_executor, termination], check_tool_calls)
        machine.connect(tool_executor, llm_processor)  # Go back to llm after tool execution
        
        return machine

    def invoke(self, query: str) -> Run:
        """
        Run the agent on a query
        
        Args:
            query: The user's query to process
            
        Returns:
            The final run object after processing
        """

        initial_state: AgentState = {
            "query": query,
            "messages": list(self.messages),
            "instructions": self.instructions,
            "current_tool_calls": None
            
        }

        run_object = self.workflow.run(initial_state)

        final_state = run_object.get_final_state()
        self.messages = final_state["messages"]

        return run_object




In [13]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?",
]

run_Agent = Agent(
    query = "",
    model_name="gpt-4o-mini",
    #messages= None,
    tools=tools,
    instructions=(
        "You're an AI Agent using internal game data (vector DB) to answer questions asked"
        "Search the web only when internal results are not enough."
        "Keep track of the conversation"
        f"Use tools only when they help. These are the tools you can use Tools: {tools}"
        "Return structured JSON when asked."
        "You're an AI Agent very good with math operations "
        "You can answer multistep questions by sequentially calling functions. "
        "You follow a pattern of of Thought and Action. "
        "Create a plan of execution: "
        "- Use Thought to describe your thoughts about the question you have been asked. "
        "- Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly."
        "Always cite your sourses. When using tool reference document or url in your final answer."
        "When you think it's over, return the answer "
        "Never try to respond directly if the question needs a tool. "
        "But if you don't have a tool available, you can respond directly. "
    )
)

workflow = StateMachine[AgentState](AgentState)


for q in questions:

    run_object = run_Agent.invoke(q)
    final_state = run_object.get_final_state()
    messages = final_state.get("messages",[])

    #get expected type into context from evaluate_tool 
    raw_docs = retrieve_game(q)
    retrieved_docs = [
    doc if isinstance(doc, str) else doc.get("Description", "")
    for doc in raw_docs
    ]
    reasoning = evaluate_retrieval(q, retrieved_docs)

    #changed print statements to not be hard coded as i was getting erros. ChatGPT helped to provide guidance / structure. 
    response = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            response = msg
            break
    last_tool = None
    for msg in reversed(messages):
        if isinstance(msg, ToolMessage):
            last_tool = msg
            break

    #print(final_state)
 
    print(f"\n- Question: {q} -")
    print("- Final Answer:", response.content)
    print("- Tool Usage", last_tool.tool_call_id)
    print("- Reasoning: ", reasoning,"\n")

    
    


[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

- Question: When Pokémon Gold and Silver was released? -
- Final Answer: Pokémon Gold and Silver was released in 1999 for the Game Boy Color. These games are notable for being the second generation of Pokémon games, introducing new regions, Pokémon, and gameplay mechanics.
- Tool Usage call_22OiKjqmemBGj9BGGrmHs9Nz
- Reasoning:  {'useful': True, 'description': 'Pokémon Gold and Silver was released in 1999.'} 

[StateMachine] Starting: __entry__
[StateMachine] Executing step: mes

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes